<a href="https://colab.research.google.com/github/shatabdi-sikta/Clinix-Link-Medical-NLP/blob/main/medical_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[senrencepiece] accelerate spacy --quiet
!python -m spacy download en_core_web_sm --quiet
print("Setup 100% complete. No external Files needed.")

In [ ]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


print("Step 1: Loading Neural Medical Model (BioBERT)... Please wait about 45 seconds.")
try:

    master_ner = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")
    print(" Model Loaded.")
except Exception as e:
    print(f" Error loading model: {e}")


clinical_notes = [
    "Patient A-102: 14-day-old infant showing skeletal rickets and acute Vitamin D deficiency.",
    "Patient B-405: 55-year-old male with chronic chest pain and history of hypertensive heart disease.",
    "Patient C-99: Symptomatic viral pneumonia leading to acute respiratory distress syndrome.",
    "Patient D-11: Early cognitive decline detected during neurological linguistic screening."
]


final_results = []
print("\nStep 2: Starting Semantic Diagnostic Mapping...")

for note in clinical_notes:

    p_id = note.split(":")[0]


    entities = master_ner(note)

    print(f"   Analyzing {p_id}...")

    for ent in entities:
        final_results.append({
            "Patient": p_id,
            "Biomarker": ent['word'],
            "Category": ent['entity_group'],
            "Confidence": round(float(ent['score']), 3)
        })


df = pd.DataFrame(final_results)


print("\nStep 3: Generating Confidence Heatmap...")
plt.figure(figsize=(12, 8))
pivot_data = df.pivot_table(index='Biomarker', columns='Patient', values='Confidence')
sns.heatmap(pivot_data, annot=True, cmap="YlGnBu", fmt=".3f", cbar_kws={'label': 'AI Confidence'})

plt.title("Clinix-Link: Neural Semantic Mapping for Early Diagnosis", fontsize=16)
plt.xlabel("Anonymized Case ID", fontsize=12)
plt.ylabel("Detected Clinical Markers", fontsize=12)
plt.show()

print("\n PROJECT COMPLETE: You can now screenshot this Heatmap for your application.")